In [27]:
%pip install seqeval

Note: you may need to restart the kernel to use updated packages.


In [28]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings('ignore')

## Work With Data

In [29]:
ua_df = pd.read_csv("/kaggle/input/kse-ua-location-extraction-2025/uk_geo_dataset.csv")
ua_df['geo_is'] = 'ua'
ru_df = pd.read_csv("/kaggle/input/kse-ua-location-extraction-2025/ru_geo_dataset.csv")
ru_df['geo_is'] = 'ru'

In [30]:
ua_df_loc = ua_df[ua_df['loc_markers'] != '[]'] # max 233,421
ua_df_nloc = ua_df[ua_df['loc_markers'] == '[]'] # max 776,579

ru_df_loc = ru_df[ru_df['loc_markers'] != '[]'] # max 2,464,436
ru_df_nloc = ru_df[ru_df['loc_markers'] == '[]'] # max 5,564,404

In [31]:
df_loc = pd.concat([ua_df_loc[:200000], ru_df_loc[:2000000]], axis=0)
df_nloc = pd.concat([ua_df_nloc[:500000], ru_df_nloc[:2000000]], axis=0)

In [32]:
def oversample_ua(df, factor):
    
    minority_df = df[df["geo_is"] == "ua"]
    majority_df = df[df["geo_is"] != "ua"]
    
    extra = minority_df.sample(
        n=len(minority_df) * (factor - 1),
        replace=True,
        random_state=42
    )
    
    df_oversampled = pd.concat([majority_df, minority_df, extra], ignore_index=True)
    df_oversampled = df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)
    return df_oversampled

In [33]:
df_loc = oversample_ua(df_loc, 10)
df_nloc = oversample_ua(df_nloc, 4)

df_loc = df_loc[["text", "loc_markers"]]
df_nloc = df_nloc[["text", "loc_markers"]]

In [34]:
# 3,600,000 - 90%
train_df = pd.concat([df_loc[:3600000], df_nloc[:3600000]], axis=0) 
# 400,000 - 10%;
val_df = pd.concat([df_loc[3600000:], df_nloc[3600000:]], axis=0)   

In [35]:
### TEMP ###
### train_df, val_df = train_df[:1000], val_df[:500] 
### TEMP ###

In [36]:
train_df.to_csv("train.csv", index=False)
val_df.to_csv("val.csv", index=False)

---
# Code Ends